# PDTB 句间关系分类 LoRA 微调实验
本实验使用 `MindSpore + mindnlp + LoRA` 在 `DeepSeek-R1-Distill-Qwen-1.5B` 模型上进行微调，任务目标是：
- 输入：一个句子（或对话内容）
- 输出：该句子属于哪一种 PDTB 分类，并解释原因。

In [ ]:
import mindnlp
import mindspore
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig
from peft import LoraConfig, TaskType, get_peft_model, PeftModel

# 检查mindnlp版本
print("MindNLP版本:", mindnlp.__version__)

In [ ]:
train_path = "/home/ma-user/work/data/train.json"
val_path = "/home/ma-user/work/data/val.json"

df_train = pd.read_json(train_path)
df_val = pd.read_json(val_path)

ds_train = Dataset.from_pandas(df_train)
ds_val = Dataset.from_pandas(df_val)

ds_train[:3]

In [ ]:
# 实例化tokenizer
tokenizer = AutoTokenizer.from_pretrained('deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', use_fast=False, trust_remote_code=True)
tokenizer

In [ ]:
MAX_LENGTH = 384

def process_func(example):
    instruction = tokenizer(
        f"<|im_start|>system\n你是PDTB文本关系分析助手<|im_end|>\n"
        f"<|im_start|>user\n{example.get('content','') + example.get('input','')}<|im_end|>\n"
        f"<|im_start|>assistant\n",
        add_special_tokens=False
    )
    response = tokenizer(f"{example.get('summary','')}", add_special_tokens=False)

    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]

    # 截断
    input_ids = input_ids[:MAX_LENGTH]
    attention_mask = attention_mask[:MAX_LENGTH]
    labels = labels[:MAX_LENGTH]

    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

tokenized_train = ds_train.map(process_func, remove_columns=ds_train.column_names)
tokenized_train

tokenized_val = ds_val.map(process_func, remove_columns=ds_val.column_names)
tokenized_val

tokenizer.decode(tokenized_train[0]['input_ids'])

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B',
    ms_dtype=mindspore.bfloat16,
    device_map=0
)

# 开启梯度检查点
model.enable_input_require_grads()

In [ ]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    inference_mode=False
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
# 定义训练超参数
args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=5,
    logging_steps=10,
    num_train_epochs=3,
    save_steps=100, 
    learning_rate=3e-5,
    save_on_each_node=True,
    fp16=True,
    gradient_checkpointing=True
)


In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    eval_dataset=tokenized_val
)

In [ ]:
# 开始训练
train_result = trainer.train()
print("训练结果:", train_result)

In [ ]:
# 保存最终模型
model.save_pretrained("./output/final_model")
tokenizer.save_pretrained("./output/final_model")